In [ ]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, Activation,Dropout
from keras.layers import LSTM
from keras.preprocessing import sequence
import keras as keras
import numpy as np

In [ ]:
data=pd.read_json('../input/train.json')

In [ ]:
cat_dict=[]
data_dict=[]

In [ ]:
data=data.drop(['id'],axis=1)

In [ ]:
top_len=0
for x in range(0,len(data)):
    l=len(data['ingredients'][x])
    if (l>top_len):
        top_len=l

In [ ]:
m=[]
for x in range(0,len(data)):
    k=''
    for f in data['ingredients'][x]:
        k+=f.lower()+' '
    m.append(k)
data['ingredients']=m

In [ ]:
cuisine_dic_t=[]

In [ ]:
for x in range(0,len(data)):
    if data['cuisine'][x] not in cuisine_dic_t:
        cuisine_dic_t.append(data['cuisine'][x])

In [ ]:
cuisine_dic=pd.DataFrame(data=cuisine_dic_t)

In [ ]:
cuisine=[]

In [ ]:
for x in range(0,len(data)):
    for s in range(0,len(cuisine_dic)):
        if cuisine_dic[0][s]==data['cuisine'][x]:
            k=[cuisine_dic.index[s]]
            cuisine.append(k)

In [ ]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data['ingredients'])

ingridients = tokenizer.texts_to_sequences(data['ingredients'])

In [ ]:
X_train=ingridients[:30000]
y_train=cuisine[:30000]
X_test=ingridients[30000:]
y_test=cuisine[30000:]

In [ ]:
X_train = tokenizer.sequences_to_matrix(X_train,mode='binary')
X_test = tokenizer.sequences_to_matrix(X_test,mode='binary')
y_train = keras.utils.to_categorical(y_train,len(cuisine_dic))
y_test = keras.utils.to_categorical(y_test,len(cuisine_dic))

In [ ]:
model = Sequential()
model.add(Dense(2048, input_shape=(len(tokenizer.word_index)+1,)))
model.add(Activation('relu'))
model.add(Dropout(0.7))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(len(cuisine_dic)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(X_test, y_test))

In [ ]:
model.evaluate(X_test, y_test,
                       batch_size=32, verbose=1)

In [ ]:
test_data=pd.read_json('../input/test.json')

In [ ]:
m=[]
for x in range(0,len(test_data)):
    k=''
    for f in test_data['ingredients'][x]:
        k+=f.lower()+' '
    m.append(k)
test_data['ingredients']=m

In [ ]:
ingridients_test = tokenizer.texts_to_sequences(test_data['ingredients'])

In [ ]:
ingridients_test=tokenizer.sequences_to_matrix(ingridients_test,mode='binary')

In [ ]:
s=model.predict_classes(ingridients_test)

In [ ]:
res_t=[]
for x in s:
    res_t.append(cuisine_dic[0][x])

In [ ]:
test_data['ingredients']=res_t

In [ ]:
test_data=test_data.rename(index=str, columns={'ingredients': "cuisine"})

In [ ]:
test_data.to_csv('result.csv', index=False)